In [1]:
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import pandas as pd
import scipy.stats as stats
from scipy import special, integrate

In [2]:
# Usage: ./main -N <N> -M <M> (-A) -R <rep_arg> -n <NMC> -b <burnin> -s <MCsteps> -L <SNR_max> -S <seed>
# N        : > 0, number of rows.
# M        : > 0, number of columns.
# A        : use adaptive lambda.
# rep_arg  : > if -A, then number of lambdas.
#          : otherwise, file containing the list of lambdas.
# NMC      : number of Monte Carlo steps in total.
# burnin   : >= 0, out of NMC, number of Monte Carlo steps for burn-in.
#            for adaptive lambda, recommended to be larger than 10000.
# MCsteps  : >0, number of Monte Carlo sweeps per exchange move.
# SNR_max  : maximum signal-to-noise ratio (> 0).
# seed     : seed for true signal (optional, default:12345).


#Compile the Monte Carlo code 
#Designate g++ compiler version with openmp and std=c++11 support
Compiler = "g++-13"
subprocess.run([Compiler, "-std=c++11", "-O2", "-march=native", "-fopenmp", "-o", "main", "src/main.cpp", "src/ReplicaExchange.cpp", "src/MF_Rad.cpp"])
subprocess.run([Compiler, "-std=c++11", "-O2", "-march=native", "-fopenmp", "-o", "pos", "src/PosteriorSample.cpp", "src/ReplicaExchange.cpp", "src/MF_Rad.cpp"])
subprocess.run([Compiler, "-std=c++11", "-O2", "-march=native", "-fopenmp", "-o", "metropolis", "src/LocalMetropolis.cpp", "src/ReplicaExchange.cpp", "src/MF_Rad.cpp"])

# Run replica exchange Monte Carlo simulation to obtain the averaged overlap matrix and its square for each SNR
def run_MC(N, M, SNR_file, NMC, MCsteps, seed, SNR_max = 20.0, burnin_rate = 0.2):
    burnin = int(burnin_rate*NMC)
    print("Running Monte Carlo simulation with N = ", N, ", M = ", M, ", NMC = ", NMC, ", MCsteps = ", MCsteps, ", seed = ", seed)
    subprocess.run(["./main", "-N", str(N), "-M", str(M), "-R", SNR_file, "-n", str(NMC), "-b", str(burnin), "-s", str(MCsteps), "-L", str(SNR_max) , "-S", str(seed)])

# Run replica exchange Monte Carlo simulation to obtain the posterior samples, given by the overlap matrix
def run_POS(N, M, SNR_file, NMC, MCsteps, seed, SNR_max = 20.0, burnin_rate = 0.8):
    burnin = int(burnin_rate*NMC)
    print("Running Monte Carlo simulation with N = ", N, ", M = ", M, ", NMC = ", NMC, ", MCsteps = ", MCsteps, ", seed = ", seed)
    subprocess.run(["./pos", "-N", str(N), "-M", str(M), "-R", SNR_file, "-n", str(NMC), "-b", str(burnin), "-s", str(MCsteps), "-L", str(SNR_max) , "-S", str(seed)])

# Run standard Metropolis Monte Carlo simulation to obtain the MMSE under informative and uninformative initialization 
def run_Metropolis(N, M, SNR_file, NMC, MCsteps, seed, rand_init, burnin_rate = 0.2):
    burnin = int(burnin_rate*NMC * MCsteps)
    real_NMC = NMC * MCsteps
    print("Running Metropolis Monte Carlo simulation with N = ", N, ", M = ", M, ", Monte Carlo Sweeps = ", real_NMC, ", seed = ", seed, "with " + ("random" if rand_init else "informative") + " initialization")
    p = 0.5 if rand_init else 0.0
    subprocess.run(["./metropolis", "-N", str(N), "-M", str(M), "-R", SNR_file, "-n", str(real_NMC), "-b", str(burnin), "-S", str(seed), "-p", str(p)])

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.size'] = 14.0
plt.rcParams['font.family'] = 'serif' 
plt.rcParams['text.usetex'] = True
plt.rc('text.latex', preamble=r'\usepackage{bm}')

plt.rcParams.update(
    {   'text.usetex': True,
        'text.latex.preamble': r'\usepackage{amsfonts} \usepackage{bm}'},
)


In [3]:
#Code to plot the MMSE and MI from Gaussian RIE
def spectral_density_Wishart(x, alpha, lambda_):
    f = ((np.sqrt(lambda_) * x - 2) * (2*lambda_*x**2 + np.sqrt(lambda_)*x - 9*lambda_ - 1) / alpha + 9 * (lambda_**1.5*x + lambda_))**2 / alpha - 4 * (lambda_ * (x**2-3) / alpha - np.sqrt(lambda_) * x / alpha + 1.0 / alpha + 3*lambda_)**3
    if f > 0:
        a = alpha**(-1.5) * (np.sqrt(lambda_)*x - 2) * (lambda_ * (2*x**2 - 9) + np.sqrt(lambda_) * x - 1) + 9*lambda_ * (np.sqrt(lambda_) * x + 1) / np.sqrt(alpha) + np.sqrt(f)
        return (np.cbrt(2*a**2) - 2 * (lambda_*(x**2 - 3) / alpha - np.sqrt(lambda_)*x / alpha + 1/alpha + 3*lambda_)) / (np.pi * 2**(5.0/3) * np.sqrt(3*lambda_/alpha) * np.cbrt(a))
    else:
        return 0.0

def Exact_MMSE(alpha, lambda_):
    Int, _ = integrate.quad(lambda x: spectral_density_Wishart(x, alpha, lambda_)**3, -5.0, 5.0 + lambda_)
    return (1.0 - Int * 4*np.pi**2 / 3) / (alpha * lambda_)

#Integrate the MMSE from 0 to lambda_ using trapezoid rule, with lambda_ divided into 100 parts by default
def MutualInformation(alpha, lambda_, delta_lambda=None):
    if delta_lambda is None:
        delta_lambda = lambda_ / 100.0
    lambda_values = np.arange(0.001, lambda_ + delta_lambda, delta_lambda)
    MI = np.zeros(len(lambda_values))
    # integrate according to trapezoid rule
    MMSE_list = [Exact_MMSE(alpha, lambda_val) for lambda_val in lambda_values]
    MMSE_list[0] = MMSE_list[1]
    for idx in range(1, len(lambda_values)):
        MI[idx] = (MMSE_list[idx] + MMSE_list[idx-1]) * delta_lambda / 2 + MI[idx-1]
    return lambda_values, MI / 4

In [4]:
#Calculate the mutual information and MMSE from the Monte Carlo simulation
def GET_AVERAGE_MMSE_MI(seed_list, N, M, NMC):
    DATA = [ np.loadtxt("MC_data/main/N" + str(N) + "_M"  + str(M) + "_seed" + str(seed) +  "_NMC" + str(NMC) + ".txt") for seed in seed_list]
    DATA = np.array(DATA)

    #Compute the mutual information and standard error
    SNRs = DATA[0, :, 0]
    MI = np.mean(DATA[:, :, 2], axis = 0)
    MI_quenched_var = np.var(DATA[:, :, 2], axis = 0)
    MI_MCMC_var = np.mean(DATA[:, :, 3]**2, axis = 0)
    MI_ste = np.sqrt(MI_quenched_var + MI_MCMC_var) / np.sqrt(len(seed_list))

    #Compute the MMSE
    MMSE = np.mean(DATA[:, :, 4], axis = 0)
    MMSE_quenched_var = np.var(DATA[:, :, 4], axis = 0)
    MMSE_MCMC_var = np.mean(DATA[:, :, 5]**2, axis = 0)
    MMSE_ste = np.sqrt(MMSE_quenched_var + MMSE_MCMC_var) / np.sqrt(len(seed_list))

    df = pd.DataFrame({"SNR": SNRs, "MI": MI, "MI_ste": MI_ste, "MMSE": MMSE, "MMSE_ste": MMSE_ste})
    return df


#Calculate the diagonal overlap and off-diagonal overlap contribution to MMSE
def GET_MMSE_CONTRIBUTION(seed_list, N, M, NMC):
    DATA = [ np.loadtxt("MC_data/overlap_sq/N" + str(N) + "_M"  + str(M) + "_seed" + str(seed) +  "_NMC" + str(NMC) + "_overlap_sq.txt") for seed in seed_list]
    DATA = np.array(DATA)

    overlap_average = np.mean(DATA[:, :, 1:], axis = 0)
    overlap_var = np.var(DATA[:, :, 1:], axis = 0)
    SNRs = DATA[0, :, 0]

    #reshape last axis to M x M
    overlap_average = overlap_average.reshape(-1, M, M)
    overlap_var = overlap_var.reshape(-1, M, M)
    #Last row corresponds to contribution from X0 
    X0_overlap = overlap_average[-1, :, :]
    X0_overlap_var = overlap_var[-1, :, :]
    MMSE_diag = np.zeros(np.size(SNRs))
    MMSE_off_diag = np.zeros(np.size(SNRs))
    MMSE_off_diag_ste = np.zeros(np.size(SNRs))
    MMSE_diag_ste = np.zeros(np.size(SNRs))

    #Compute the contribution from diagonal and off-diagonal elements
    for r in range(np.size(SNRs)):
        MMSE_diag[r] = -np.sum(np.diag(overlap_average[r, :, :])) + np.sum(np.diag(X0_overlap))
        MMSE_off_diag[r] = -np.sum(overlap_average[r, :, :]) + np.sum(X0_overlap) + np.sum(np.diag(overlap_average[r, :, :])) - np.sum(np.diag(X0_overlap))
        #Variance from diagonal and off-diagonal elements
        MMSE_diag_var = np.sum(np.diag(overlap_var[r, :, :])) + np.sum(np.diag(X0_overlap_var))
        MMSE_off_diag_var = np.sum(overlap_var[r, :, :]) + np.sum(X0_overlap_var) + np.sum(np.diag(overlap_var[r, :, :])) + np.sum(np.diag(X0_overlap_var))
        MMSE_diag_ste[r] = np.sqrt(MMSE_diag_var) / np.sqrt(len(seed_list))
        MMSE_off_diag_ste[r] = np.sqrt(MMSE_off_diag_var) / np.sqrt(len(seed_list))
    
    df = pd.DataFrame({"SNR": SNRs, "MMSE_diag": MMSE_diag / M, "MMSE_diag_ste": MMSE_diag_ste / M,
                        "MMSE_off_diag": MMSE_off_diag / M , "MMSE_off_diag_ste": MMSE_off_diag_ste / M})
    return df

#Calculate the MMSE from the Metropolis Monte Carlo simulation
def GET_METROPOLIS_MMSE(seed_list, N, M, NMC, rand_init):
    real_NMC = 30 * NMC
    prob = "0.500000" if rand_init else "0.000000"
    DATA = [ np.loadtxt("Metropolis_data/N" + str(N) + "_M"  + str(M) + "_seed" + str(seed) +  "_NMC" + str(real_NMC) + "prob" + str(prob) + ".txt") for seed in seed_list]
    DATA = np.array(DATA)

    #Compute the MMSE
    MMSE = np.mean(DATA[:, :, 1], axis = 0)
    MMSE_quenched_var = np.var(DATA[:, :, 1], axis = 0)
    MMSE_MCMC_var = np.mean(DATA[:, :, 2]**2, axis = 0)
    MMSE_ste = np.sqrt(MMSE_quenched_var + MMSE_MCMC_var) / np.sqrt(len(seed_list))

    df = pd.DataFrame({"SNR": DATA[0, :, 0], "MMSE": MMSE, "MMSE_ste": MMSE_ste})
    return df

In [ ]:
#Run Monte Carlo simulation over several seeds 
N = 20; M = 10; NMC = 20000; steps = 30; seedlist = range(1)
for seed in seedlist:
    run_MC(N, M, "lambda_list/lambda_list_N20_alpha0.5.txt", NMC, steps, seed, burnin_rate=0.5)

In [6]:
df = GET_AVERAGE_MMSE_MI(seedlist, N, M, NMC)

In [ ]:
plt.title("Mutual Information")
plt.errorbar(df["SNR"], df["MI"], yerr = df["MI_ste"], label = "$N =" + str(N) + " ,M = " + str(M) + "$" , marker = "o", ecolor='gray', markersize = 3)

x, MI = MutualInformation(0.5, 20.0)
plt.plot(x, MI, label = "Gaussian RIE", color = "red")

plt.legend()
plt.show()
plt.title("MMSE")
plt.errorbar(df["SNR"], df["MMSE"], yerr = df["MMSE_ste"], label = "$N =" + str(N) + " ,M = " + str(M) + "$" , marker = "o", ecolor='gray', markersize = 3)
SNRs = np.linspace(0.01, 20.0, 100)
MMSE = [Exact_MMSE(0.5, SNR) for SNR in SNRs]
plt.plot(SNRs, MMSE, label = "Gaussian RIE", color = "red")
plt.legend()
plt.show()

In [ ]:
df = GET_MMSE_CONTRIBUTION(seedlist, N, M, NMC)
plt.errorbar(df["SNR"], df["MMSE_diag"], yerr = df["MMSE_diag_ste"], label = "MMSE_diag", marker = "o", ecolor='gray', markersize = 3)
plt.errorbar(df["SNR"], df["MMSE_off_diag"], yerr = df["MMSE_off_diag_ste"], label = "MMSE_off_diag", marker = "o", ecolor='gray', markersize = 3)
plt.legend()

In [12]:
def GET_OVERLAP_SQ_AVERAGE(seed_list, N, M , NMC):
    DATA = [ np.loadtxt("MC_data/overlap_sq/N" + str(N) + "_M"  + str(M) + "_seed" + str(seed) +  "_NMC" + str(NMC) + "_overlap_sq.txt") for seed in seed_list]
    DATA = np.mean(np.array(DATA), axis = 0)
    #reshape last axis to M x M 
    overlap_sq = DATA[:, 1:].reshape(-1, M, M)
    SNRs = DATA[:, 0]
    return SNRs, overlap_sq

def GET_CLOSEST_SNRs(lambdas_target, lambda_list):
    index_list = [ np.argmin(np.abs(lambdas - lambda_list)) for lambdas in lambdas_target]
    return index_list, [lambda_list[index] for index in index_list]

    


In [ ]:
lambdas, overlap_sq = GET_OVERLAP_SQ_AVERAGE(seedlist, N, M, NMC)
lambda_list = np.linspace(3.0, 8.0, 9)
print(lambda_list)
index_list, lambda_list = GET_CLOSEST_SNRs(lambda_list, lambdas)
fig, axes = plt.subplots(3, 3, figsize=(13, 13))
for i, ax in enumerate(axes.flat):
    lambda_index = index_list[i]
    im = ax.imshow(overlap_sq[lambda_index,:,:], cmap='hot', vmin=0, vmax=1)
    ax.set_title(f'$\lambda = {lambdas[lambda_index]:.3f}$')
    ax.set_xticks(np.arange(0, M, 5))
    ax.set_yticks(np.arange(0, M, 5))
cbar_ax = fig.add_axes([0.93, 0.11, 0.05, 0.77])
fig.colorbar(im, cax=cbar_ax)
plt.suptitle(r"$ \displaystyle \frac{1}{N^2} \left \langle (\bm{X}^\mathsf{T} \bm{x})^2 \right\rangle \in \mathbb{R}^{N\times N} $")
#plt.savefig("overlap_sq_alpha" + alpha_str + "_matrix.pdf", bbox_inches='tight')


In [ ]:
#Accumulate the Posterior samples for a single instance of X and Z 
poseed = 1
NMC_pos = 20000
N=60
M=30
run_POS(N, M, "lambda_list/lambda_list_N60_alpha0.5_PosteriorSamples.txt", NMC_pos, steps, poseed, burnin_rate=0.5)
# Since the task is to obtain the posteror samples, rather than to estimate the mutual information, 
# The SNRs do not need to be such that they are spaced with finite exchange rate; they only need to be so near the transition point. 
# Note that for low SNR (SNR = 0~4) , local spin flip moves are sufficient to obtain the samples (see original paper). 

In [25]:
def GET_POSTERIOR_SAMPLES(N, M, NMC, seed, SNR):
    SNR_str = "{:.4f}".format(SNR)
    DATA = np.loadtxt("MC_data_samples/N" + str(N) + "_M"  + str(M) + "_seed" + str(seed) +  "_NMC" + str(NMC) + "_lambda" + SNR_str + ".txt")
    return DATA

def GET_OPTIMAL_ROTATED_POSTERIOR_SAMPLES(N, M, NMC, seed, SNR):
    SNR_str = "{:.4f}".format(SNR)
    DATA = np.loadtxt("MC_data_samples/N" + str(N) + "_M"  + str(M) + "_seed" + str(seed) +  "_NMC" + str(NMC) + "_lambda" + SNR_str + ".txt")
    sample_size = np.size(DATA, 0)
    rotated_overlap_average = np.zeros((M, M))
    for i in range(sample_size):
        X = DATA[i,:].reshape(M,M) / (2.0*M)
        U, S, _ = np.linalg.svd(X)
        rotated_overlap_average += U @ np.diag(S) @ U.transpose() / sample_size

    return rotated_overlap_average

In [ ]:
lambdas = np.loadtxt("lambda_list/lambda_list_N60_alpha0.5_PosteriorSamples.txt")
index_list, SNR_list = GET_CLOSEST_SNRs([0.5, 1.0, 2.0, 4.0, 6.0], lambdas)

#Get reference distribution: random Rademacher 
rv = stats.binom(N, 0.5)
x = np.arange(0,N+1,1)
hist_ = rv.pmf(x)
bins = 2 * ( x-M) / np.sqrt(N)


plt.hist(bins, N+1, weights=hist_, histtype = "stepfilled", edgecolor = "black",
         color='lightgrey', label = 'random patterns', linestyle='dashed', lw = 2, alpha = 0.6, 
         density=True
         )
for SNR in SNR_list:
    DATA = GET_POSTERIOR_SAMPLES(N, M, NMC_pos, poseed, SNR).ravel()
    plt.hist(DATA / np.sqrt(N) , bins = bins + 1.0 / np.sqrt(N), density = True, histtype=u"step", 
             label = r'$\lambda = $' + "{:.4f}".format(SNR), linewidth = 2)

plt.legend()


In [ ]:
#Plot OptimalRotation_overlap for each alpha in heatmap 
index_list, SNR_list = GET_CLOSEST_SNRs([0.0,2.0,4.0, 6.0, 6.2, 6.5, 7.0, 7.5, 12.0], lambdas)

fig, ax =plt.subplots(3, 3, figsize=(16, 16))

tot_idx = 0
for SNR in SNR_list:
    R= GET_OPTIMAL_ROTATED_POSTERIOR_SAMPLES(N, M, NMC_pos, poseed, SNR)
    col_idx = tot_idx // 3
    row_idx = tot_idx % 3
    im = ax[col_idx, row_idx ].imshow(R, cmap='hot', vmin = 0.0, vmax = 1.0)
    ax[col_idx, row_idx].set_title(r"$\lambda = " + "{:.4f}".format(SNR) + "$")
    tot_idx += 1
    #fig.colorbar(im, ax=ax[col_idx, row_idx])
cbar_ax = fig.add_axes([0.93, 0.11, 0.05, 0.77])
fig.colorbar(im, cax=cbar_ax)

In [ ]:
N=40
M=28
NMC_pos = 10000
seedlist = range(4)
for seed in seedlist:
    run_Metropolis(N, M, "lambda_list/lambda_list_N40_alpha0.7_Metropolis.txt", NMC_pos, steps, seed, rand_init = True, burnin_rate=0.2)
    run_Metropolis(N, M, "lambda_list/lambda_list_N40_alpha0.7_Metropolis.txt", NMC_pos, steps, seed, rand_init = False, burnin_rate=0.2)

In [ ]:
#Plot MSE for random and informative initialization
df_random = GET_METROPOLIS_MMSE(seedlist, N, M, NMC_pos, True)
df_informative = GET_METROPOLIS_MMSE(seedlist, N, M, NMC_pos, False)

plt.errorbar(df_random["SNR"], df_random["MMSE"], yerr = df_random["MMSE_ste"], label = "Random Initialization", marker = "o", ecolor='gray', markersize = 3)
plt.errorbar(df_informative["SNR"], df_informative["MMSE"], yerr = df_informative["MMSE_ste"], label = "Informative Initialization", marker = "o", ecolor='gray', markersize = 3)

SNRs = np.linspace(0.01, 20.0, 100)
MMSE = [Exact_MMSE(0.7, SNR) for SNR in SNRs]
plt.plot(SNRs, MMSE, label = "Gaussian RIE", linestyle = "dashed")

plt.legend()
